In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import re
print('##################################')
print('#       Preprocessing            #')
print('##################################')
lines = open('/content/movie_lines.txt', encoding='utf-8',errors='ignore').read().split('\n')
#errors will ignore the error
convers = open('/content/movie_conversations.txt', encoding='utf-8',errors='ignore').read().split('\n')



# We need only coversation and their unique_id
exchn = []
for conver in convers:
    exchn.append(conver.split(' +++$+++ ')[-1][1:-1].replace("'", " ").replace(",","").split())
    


# We need the sequence of the conversation
#Creating list with question with answer
diag = {}
for line in lines:
    diag[line.split(' +++$+++ ')[0]] = line.split(' +++$+++ ')[-1]


questions = []
answers = []

for conver in exchn:
    for i in range(len(conver) - 1):
        questions.append(diag[conver[i]])
        answers.append(diag[conver[i+1]])

##################################
#       Preprocessing            #
##################################


In [5]:
# question and there answer
questions[0],answers[0]

('Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.")

In [6]:
# Creating a fixed length code i.e 13
sorted_ques = []
sorted_ans = []
for i in range(len(questions)):
    if len(questions[i]) < 13:
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])


# changing the text with there
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt


# Cleaning Punctuation and Cleaned
clean_ques = []
clean_ans = []

for line in sorted_ques:
    clean_ques.append(clean_text(line))
        
for line in sorted_ans:
    clean_ans.append(clean_text(line))






for i in range(len(clean_ans)):
    clean_ans[i] = ' '.join(clean_ans[i].split()[:11])

In [7]:
clean_ans[0]

'the thing is cameron i am at the mercy of a'

In [8]:
## trimming
clean_ans=clean_ans[:30000]
clean_ques=clean_ques[:30000]



###  count occurences less occurance ###
word2count = {}

for line in clean_ques:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in clean_ans:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1




###  remove less frequent ###
thresh = 5

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1

In [9]:
# Add SOS and EOS
for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'



tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1
    
    

vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0



### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}


encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)


In [10]:
clean_ans[0],encoder_inp[0]

('<SOS> the thing is cameron i am at the mercy of a <EOS>', [3024])

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Padding
encoder_inp = pad_sequences(encoder_inp, 13, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 13, padding='post', truncating='post')




decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 13, padding='post', truncating='post')




from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [12]:
encoder_inp.shape

(30000, 13)

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input


enc_inp = Input(shape=(13, ))
dec_inp = Input(shape=(13, ))


VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=13,
                  trainable=True                  
                  )


enc_embed = embed(enc_inp)
enc_lstm = LSTM(400, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]



dec_embed = embed(dec_inp)
dec_lstm = LSTM(400, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

dense_op = dense(dec_op)

model = Model([enc_inp, dec_inp], dense_op)




model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=50)

Epoch 1/50
938/938 [==============================] - 263s 275ms/step - loss: 3.0864 - acc: 0.4948
Epoch 2/50
938/938 [==============================] - 258s 275ms/step - loss: 2.7274 - acc: 0.5328
Epoch 3/50
938/938 [==============================] - 257s 275ms/step - loss: 2.6014 - acc: 0.5429
Epoch 4/50
938/938 [==============================] - 258s 275ms/step - loss: 2.5270 - acc: 0.5479
Epoch 5/50
938/938 [==============================] - 259s 276ms/step - loss: 2.4691 - acc: 0.5509
Epoch 6/50
938/938 [==============================] - 257s 274ms/step - loss: 2.4198 - acc: 0.5535
Epoch 7/50
938/938 [==============================] - 255s 272ms/step - loss: 2.3741 - acc: 0.5559
Epoch 8/50
938/938 [==============================] - 260s 277ms/step - loss: 2.3290 - acc: 0.5583
Epoch 9/50
938/938 [==============================] - 257s 274ms/step - loss: 2.2844 - acc: 0.5606
Epoch 10/50
938/938 [==============================] - 257s 273ms/step - loss: 2.2424 - acc: 0.5622
Epoch 11/

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import numpy as np

enc_model = Model([enc_inp], enc_states)



# decoder Model
decoder_state_input_h = Input(shape=(400,))
decoder_state_input_c = Input(shape=(400,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]


dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)




from keras.preprocessing.sequence import pad_sequences
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    prepro1  = input("you : ")
    ## prepro1 = "Hello"

    prepro1 = clean_text(prepro1)
    ## prepro1 = "hello"

    prepro = [prepro1]
    ## prepro1 = ["hello"]

    txt = []
    for x in prepro:
        # x = "hello"
        lst = []
        for y in x.split():
            ## y = "hello"
            try:
                lst.append(vocab[y])
                ## vocab['hello'] = 454
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    ## txt = [[454]]
    txt = pad_sequences(txt, 13, padding='post')

    ## txt = [[454,0,0,0,.........13]]

    stat = enc_model.predict( txt )

    empty_target_seq = np.zeros( ( 1 , 1) )
     ##   empty_target_seq = [0]


    empty_target_seq[0, 0] = vocab['<SOS>']
    ##    empty_target_seq = [255]

    stop_condition = False
    decoded_translation = ''

    while not stop_condition :

        dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
        decoder_concat_input = dense(dec_outputs)
        ## decoder_concat_input = [0.1, 0.2, .4, .0, ...............]

        sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
        ## sampled_word_index = [2]

        sampled_word = inv_vocab[sampled_word_index] + ' '

        ## inv_vocab[2] = 'hi'
        ## sampled_word = 'hi '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
            stop_condition = True 

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        ## <SOS> - > hi
        ## hi --> <EOS>
        stat = [h, c]  

    print("chatbot attention : ", decoded_translation )
    print("==============================================")  

##########################################
#       start chatting ver. 1.0          #
##########################################
you : hey
chatbot attention :  hey 
you : whatsup?
chatbot attention :  <OUT> 
you : suup?
chatbot attention :  <OUT> 
you : what are you doing
chatbot attention :  i am not sure it would be nice to spend a 
you : tell me some thing about
chatbot attention :  i am not going to i am going to let you 
you : are you chatboit
chatbot attention :  what 
you : are you chatbot
chatbot attention :  what 
you : are you human being
chatbot attention :  <OUT> 
you : what are you doing?
chatbot attention :  i am not sure it would be nice to spend a 
you : did you eat food?
chatbot attention :  no 
you : why didnt you eat?
chatbot attention :  i am not sure 
you : why are not sure?
chatbot attention :  i dont know 
you : tell me why 
chatbot attention :  well i am going to say one of your <OUT> to 
you : this is not right
chatbot attention :  what 
you : you are a good bo